# Informationen für dieses Notebook

Dieses .ipnynb dient dazu, um den Datensatz zu encoden und zu scalen, um eine similarity-matrix zu erstellen, welche danach in dem .ipynb "suggestion_rating_prediction" genutzt werden kann, um Filmvorschläge zu machen, welche der Nutzer kennt und welche er dann individuell bewerten kann.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast  # Um Strings wieder in Listen zu konvertieren
import random

In [2]:
%matplotlib inline
sns.set_theme(style="darkgrid", palette="bright", context="paper")

In [3]:
# Maximale Anzahl an Spalten und Breite festlegen
pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)  # Breite des Outputs in Zeichen

In [4]:
# CSV-Datei aus dem Ordner Datasets laden
df_movie_suggestion = pd.read_csv('cleaned_movie_suggestion.csv')

# CSV einlesen
df_movie_suggestion = pd.read_csv("cleaned_movie_suggestion.csv")

# Spaltennamen in Kleinbuchstaben umwandeln
df_movie_suggestion.columns = df_movie_suggestion.columns.str.lower()

# Listen wiederherstellen
list_columns = ['directors', 'writers', 'stars', 'genres', 'countries_origin', 'languages', 'production_companies']
for col in list_columns:
    df_movie_suggestion[col] = df_movie_suggestion[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# DataFrame anzeigen 
df_movie_suggestion.head(5)

,title,year,duration,rating,directors,writers,stars,genres,countries_origin,production_companies,languages,wins,nominations,oscars,mpa_category
0,Jaws,1975,124.0,8.1,[Steven Spielberg],"[Peter Benchley, Carl Gottlieb]","[Roy Scheider, Robert Shaw, Richard Dreyfuss]","[Horror, Abenteuer, Drama, Thriller]",[United States],"[Zanuck/Brown Productions, Universal Pictures]",[English],0,20,0,Parental Guidance (+13)
1,The Rocky Horror Picture Show,1975,100.0,7.4,[Jim Sharman],"[Richard O'Brien, Jim Sharman]","[Tim Curry, Susan Sarandon, Barry Bostwick]","[Horror, Komödie, Musik und Tanz]","[United Kingdom, United States]","[Twentieth Century Fox, Michael White Producti...",[English],0,4,0,Mature Audiences (+18)
2,One Flew Over the Cuckoo's Nest,1975,133.0,8.7,[Milos Forman],"[Lawrence Hauben, Bo Goldman, Ken Kesey]","[Jack Nicholson, Louise Fletcher, Michael Berr...",[Drama],[United States],"[Fantasy Films, N.V. Zvaluw]",[English],0,15,0,Mature Audiences (+18)
3,Dog Day Afternoon,1975,125.0,8.0,[Sidney Lumet],"[Frank Pierson, P.F. Kluge, Thomas Moore]","[Al Pacino, John Cazale, Penelope Allen]","[Krimi, Drama, Thriller]",[United States],"[Warner Bros., Artists Entertainment Complex]",[English],0,20,0,Mature Audiences (+18)
4,Shampoo,1975,110.0,6.4,[Hal Ashby],"[Robert Towne, Warren Beatty]","[Warren Beatty, Julie Christie, Goldie Hawn]","[Drama, Komödie]",[United States],"[Persky-Bright / Vista, Columbia Pictures, Rub...",[English],0,11,0,Mature Audiences (+18)


In [ ]:
# Backup vor dem Encoding erstellen
df_movie_pre_encoding = df_movie_suggestion.copy()

# Sicherstellen, dass die Kopie erfolgreich gespeichert wurde
print("✅ Backup von df_movie_suggestion gespeichert!")

✅ Backup von df_movie_suggestion gespeichert!


# Encoding

In [12]:
import ast

# Falls 'genres' als String gespeichert wurde, zurück in Listen umwandeln
df_movie_suggestion['genres'] = df_movie_suggestion['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Falls noch `nan`-Werte vorhanden sind, ersetze sie durch leere Listen
df_movie_suggestion['genres'] = df_movie_suggestion['genres'].apply(lambda x: x if isinstance(x, list) else [])

# Test: Sind alle Genres Listen?
print(df_movie_suggestion['genres'].apply(type).value_counts())


genres
<class 'list'>    30867
Name: count, dtype: int64


Es wären zu viele Features wenn man alles encoded. Daher Top_K Ansatz mit folgenden Werten:

In [13]:
top_k = {
    'directors': 500,       # Top 500 Regisseure
    'writers': 1000,        # Top 1000 Autoren
    'stars': 1000,          # Top 1000 Schauspieler
    'genres': None,         # Alle Genres (keine Reduktion nötig)
    'countries_origin': 20, # Top 20 Länder
    'languages': 20         # Top 20 Sprachen
}


Code zum Encoden

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Verarbeitung der Spalten

# 1. Multi-Label-Spalten (genres, countries_origin, languages)
multi_label_cols = ['genres', 'countries_origin', 'languages']
for col in multi_label_cols:
    mlb = MultiLabelBinarizer()
    if top_k[col]:  # Top-K Filtern
        top_values = df_movie_pre_encoding[col].explode().value_counts().index[:top_k[col]]
        df_movie_pre_encoding[col] = df_movie_pre_encoding[col].apply(lambda x: [v for v in x if v in top_values])

    # Multi-Hot-Encoding
    encoded = pd.DataFrame(mlb.fit_transform(df_movie_pre_encoding[col]), columns=mlb.classes_)
    df_movie_pre_encoding = pd.concat([df_movie_pre_encoding, encoded], axis=1)
    df_movie_pre_encoding.drop(columns=[col], inplace=True)

# 2. Textbasierte Spalten mit TF-IDF (directors, writers, stars, production_companies)
text_cols = ['directors', 'writers', 'stars', 'production_companies']
for col in text_cols:
    # Listen in Strings umwandeln, wobei die Namen durch Kommas getrennt werden
    df_movie_pre_encoding[col] = df_movie_pre_encoding[col].apply(lambda x: ','.join(x))
    
    # TF-IDF anwenden, wobei vollständige Namen als Einheit behandelt werden
    tfidf = TfidfVectorizer(max_features=top_k.get(col, 100), token_pattern=r'[^,]+')  # Trennung anhand von Kommas
    tfidf_matrix = tfidf.fit_transform(df_movie_pre_encoding[col])
    
    # DataFrame mit den neuen Features erstellen
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f"{col}_{feat}" for feat in tfidf.get_feature_names_out()])
    
    # Kontrolle der neuen Spalten
    print(f"✍️ Kontrolle {col}: {len(tfidf_df.columns)} Features erstellt")
    
    # Hinzufügen der kodierten Spalten und Löschen der Original-Spalte
    df_movie_pre_encoding = pd.concat([df_movie_pre_encoding, tfidf_df], axis=1)
    df_movie_pre_encoding.drop(columns=[col], inplace=True)


# One-Hot-Encoding für die Spalte mpa_category
ohe = OneHotEncoder(sparse_output=False, drop='first')  # Aktualisiert: sparse_output anstelle von sparse
mpa_encoded = ohe.fit_transform(df_movie_pre_encoding[['mpa_category']])
mpa_encoded_df = pd.DataFrame(mpa_encoded, columns=[f"mpa_{cat}" for cat in ohe.categories_[0][1:]])

# Kombinieren mit dem ursprünglichen DataFrame
df_movie_pre_encoding = pd.concat([df_movie_pre_encoding, mpa_encoded_df], axis=1)
df_movie_pre_encoding.drop(columns=['mpa_category'], inplace=True)

# 4. Numerische Spalten beibehalten (wins, nominations, oscars)
# Diese bleiben unverändert im DataFrame

# Ausgabe der finalen Spaltenanzahl
print(f"🔢 Anzahl der Spalten nach vollständigem Encoding: {df_movie_pre_encoding.shape[1]}")



✍️ Kontrolle directors: 500 Features erstellt
✍️ Kontrolle writers: 1000 Features erstellt
✍️ Kontrolle stars: 1000 Features erstellt
✍️ Kontrolle production_companies: 100 Features erstellt
🔢 Anzahl der Spalten nach vollständigem Encoding: 2671


In [ ]:
df_test_similarity = df_movie_pre_encoding.copy()
print("✅ Test-DataFrame für Similarity-Analyse erstellt!")

✅ Test-DataFrame für Similarity-Analyse erstellt!


In [ ]:
df_test_similarity.to_csv("df_test_similarity.csv", index=False)
print("✅ DataFrame wurde als 'df_test_similarity.csv' gespeichert!")

✅ DataFrame wurde als 'df_test_similarity.csv' gespeichert!


### Skalieren für ausgewählte Parameter

In [18]:
from sklearn.preprocessing import StandardScaler

# Numerische Spalten, die skaliert werden sollen
num_cols = ['year', 'duration', 'rating', 'wins', 'nominations', 'oscars']

# StandardScaler anwenden
scaler = StandardScaler()
df_test_similarity_scaled = df_test_similarity.copy()
df_test_similarity_scaled[num_cols] = scaler.fit_transform(df_test_similarity_scaled[num_cols])

print("✅ Numerische Spalten erfolgreich skaliert und im DataFrame 'df_test_similarity_scaled' gespeichert!")


✅ Numerische Spalten erfolgreich skaliert und im DataFrame 'df_test_similarity_scaled' gespeichert!


In [20]:
# DataFrame als CSV speichern
df_test_similarity_scaled.to_csv('df_test_similarity_scaled.csv', index=False)

print("Die Datei wurde erfolgreich als 'df_test_similarity_scaled.csv' gespeichert!")

Die Datei wurde erfolgreich als 'df_test_similarity_scaled.csv' gespeichert!


### Similarity-Matrix (Ähnlichkeitsmatrix) erstellen

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Auswahl der relevanten Spalten (ohne 'title')
cols_for_similarity = df_test_similarity_scaled.columns.difference(['title'])

# Sparse-Matrix erstellen
sparse_data = csr_matrix(df_test_similarity_scaled[cols_for_similarity].to_numpy())

# Cosine Similarity berechnen
cosine_similarity_matrix = cosine_similarity(sparse_data, dense_output=False)

# Ausgabe der Matrix (optional)
print("✅ Sparse Cosine Similarity-Matrix erfolgreich berechnet!")



✅ Sparse Cosine Similarity-Matrix erfolgreich berechnet!


Similarity-Matrix speichern, damit man es nicht immer neu berechnen muss

In [ ]:
import pickle

# Cosine Similarity-Matrix speichern mit Protocol 4
with open("similarity_matrix.pkl", "wb") as f:
    pickle.dump(cosine_similarity_matrix, f, protocol=4)

print("✅ Cosine Similarity-Matrix wurde erfolgreich gespeichert!")

✅ Cosine Similarity-Matrix wurde erfolgreich gespeichert!


### Weiter gehts im .ipynb "03_suggestion_rating_prediction"